# Laboratorio 5 - Parte 1

### Máquinas de Vectores de Soporte

### 2018-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante:
Alejandro Castaño Rojas
#### Segundo Integrante:
Angélica Arroyave Mendoza

In [1]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")
%matplotlib inline

## Ejercicio 1

En este ejercicio deben resolver un problema de clasificación multi-clase usando una SVM. Teniendo en cuenta que la formulación original de la SVM sólo permite resolver problemas bi-clase, deben comparar los resultados obtenidos usando una estrategia **Uno vs Uno** con una estrategia **Uno vs El resto**.

Cargamos los datos:

In [3]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

Consutar el manejo de la librería sklearn para entrenar un modelos SVM en: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC. Para el caso multiclase la librería ya tiene implementada la estrategia **Uno vs Uno**, así que en este caso sólo deben llamar correctamente los métodos. Complete el siguiente código usando un clasificador basado en SVM:

In [4]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold

def SVM_defined_function(kernel, C, gamma):
    if gamma == 0:
        gamma = 'auto'
    #Validamos el modelo
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        global Xtrain
        Xtrain = X[train,:]
        global Ytrain
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        modelo = SVC(kernel= kernel, C=C, gamma=gamma)
        modelo.fit(Xtrain, Ytrain)

        #Validación
        Ytrain_pred = modelo.predict(Xtrain)
        Yest = modelo.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1
    print('kernel:',kernel,'| C:', C, '| gamma:', gamma)
    eficiencia_train = np.mean(EficienciaTrain)
    eficiencia_val = np.mean(EficienciaVal)
    percentageThatRMachines = modelo.support_vectors_.shape[0]/Xtrain.shape[0]
    std_train = np.std(EficienciaTrain)
    std_val = np.std(EficienciaVal)
    print('Eficiencia durante el entrenamiento = ' + str(eficiencia_train) + '+-' + str(std_train))
    print('Eficiencia durante la validación = ' + str(eficiencia_val) + '+-' + str(std_val))
    print('Porcentaje de muestras que son máquinas de soporte:', percentageThatRMachines)
    return eficiencia_val, std_val, percentageThatRMachines

Realice los experimientos necesarios para llenar la siguiente tabla:

In [11]:
import pandas as pd
import qgrid
kernel_list = ['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']
c_list = [0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]
gamma_list = [0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1]
df_types = pd.DataFrame({
    'Kernel' : pd.Series(kernel_list),
    'C' : pd.Series(c_list),
    'gamma' : pd.Series(gamma_list)})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
df_types["Eficiencia en validacion"][3] = "0.97077"
df_types["Intervalo de confianza"][3] = "0.01548"
df_types["% de Vectores de Soporte"][3] = "0.2620"
grid_index = 0
for kernel, c, gamma in zip(kernel_list, c_list, gamma_list):
    eficiencia, idc, ptrm = SVM_defined_function(kernel, c, gamma)
    df_types["Eficiencia en validacion"][grid_index] = eficiencia
    df_types["Intervalo de confianza"][grid_index] = idc
    df_types["% de Vectores de Soporte"][grid_index] = ptrm
    grid_index += 1
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

kernel: linear | C: 0.001 | gamma: auto
Eficiencia durante el entrenamiento = 0.9226595902460473+-0.01797959849011947
Eficiencia durante la validación = 0.7960012912715104+-0.053296257064013926
Porcentaje de muestras que son máquinas de soporte: 1.0
kernel: linear | C: 0.01 | gamma: auto
Eficiencia durante el entrenamiento = 0.9967600864350297+-0.0007994779941787222
Eficiencia durante la validación = 0.9706994383727887+-0.029264745294069673
Porcentaje de muestras que son máquinas de soporte: 0.6033210332103321
kernel: linear | C: 0.1 | gamma: auto
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9804458869759817+-0.01661113527781064
Porcentaje de muestras que son máquinas de soporte: 0.26383763837638374
kernel: linear | C: 1 | gamma: auto
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9707771243055832+-0.015485734683194906
Porcentaje de muestras que son máquinas de soporte: 0.25461254612546125
kernel: linear | C: 1

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [6]:
qgrid_widget.get_changed_df()

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
linear 0.001   0.00                  0.796001              0.0532963   
       0.010   0.00                  0.970699              0.0292647   
       0.100   0.00                  0.980446              0.0166111   
       1.000   0.00                  0.970777              0.0154857   
       10.000  0.00                  0.970777              0.0154857   
       100.000 0.00                  0.970777              0.0154857   
rbf    0.001   0.01                  0.315623               0.108064   
               0.10                  0.277914              0.0427515   
               1.00                  0.254171             0.00171694   
       0.010   0.01                  0.315623               0.108064   
               0.10                  0.277914              0.0427515   
               1.00                  0.254171             0.00171694   
       0.100   0.01                  0.832031              0.0364481   
               0.10                  0.277914              0.0427515   
               1.00                  0.254171             0.00171694   
       1.000   0.01                  0.984659              0.0115213   
               0.10                  0.808549              0.0407067   
               1.00                  0.256964             0.00648594   
       10.000  0.01                  0.990246             0.00831727   
               0.10                  0.815471              0.0376978   
               1.00                  0.258369             0.00602608   
       100.000 0.01                  0.990246             0.00831727   
               0.10                  0.815471              0.0376978   
               1.00                  0.258369             0.00602608   

                     % de Vectores de Soporte  
Kernel C       gamma                           
linear 0.001   0.00                         1  
       0.010   0.00                  0.603321  
       0.100   0.00                  0.263838  
       1.000   0.00                  0.254613  
       10.000  0.00                  0.254613  
       100.000 0.00                  0.254613  
rbf    0.001   0.01                         1  
               0.10                         1  
               1.00                  0.998155  
       0.010   0.01                         1  
               0.10                         1  
               1.00                         1  
       0.100   0.01                         1  
               0.10                         1  
               1.00                         1  
       1.000   0.01                  0.640221  
               0.10                   0.97417  
               1.00                         1  
       10.000  0.01                  0.570111  
               0.10                  0.972325  
               1.00                         1  
       100.000 0.01                  0.570111  
               0.10                  0.972325  
               1.00                         1

## Ejercicio 2

Cree dos funciones, una para entrenar un conjunto de modelos bajo la estrategia Uno vs el resto, usando como clasificador base una SVM. La segunda función debe usar el conjunto de modelos entrenados, y clasificar un conjunto de muestras de validación.

In [136]:
# Desde la librería
from sklearn.multiclass import OneVsRestClassifier
def TrainSVM_OneVsRest_Lib(X, Y, gamma, C, kernel):
    if gamma == 0:
        gamma = 'auto'
    modelo = SVC(kernel= kernel, C=C, gamma=gamma, probability=True)
    return OneVsRestClassifier(modelo, -1).fit(X, Y)

def ValidaSVM_OnevsRest_Lib(models, Xtest):
    return models.predict(Xtest)

# De
def TrainSVM_OnevsRest(X, Y, gamma, C, kernel):
    unique_classes = np.unique(Y)
    group_of_models = []
    for clss in unique_classes:
        Y_alt = Y.copy()
        Y_alt[Y == clss] = 1
        Y_alt[Y != clss] = -1
        if gamma == 0:
            gamma = 'auto'
        modelo = SVC(kernel= kernel, C=C, gamma=gamma, probability=True)
        modelo.fit(X, Y_alt)
        group_of_models.append(modelo)
    return group_of_models

def ValidaSVM_OnevsRest(models, Xtest):
    Y_prob = []
    muestra_indexer = 0
    for muestra in Xtest:
        arr_of_probs = []
        for model in models:
            arr_of_probs.append(model.predict_proba(Xtest[muestra_indexer].reshape(1, -1))[0][1])
        Y_prob.append(arr_of_probs)
        muestra_indexer += 1
    Y_est = []
    #print('yprob', Y_prob)
    for probabilities in Y_prob:
        Y_est.append(np.argmax(probabilities))
    return np.array(Y_est)
    

## Ejercicio 3

Use las funciones definidas en el punto anterior para llevar a cabo la prueba de simulación con el mismo conjunto de datos del punto 1.

In [129]:
def run_through_tests(kernel, C, gamma):
    if gamma == 0:
        gamma = 0
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento

        modelo = TrainSVM_OnevsRest(Xtrain, Ytrain, gamma=gamma, C=C, kernel=kernel)

        #Validación
        Ytrain_pred = ValidaSVM_OnevsRest(modelo, Xtrain)
        Yest = ValidaSVM_OnevsRest(modelo, Xtest)
        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1
    eficiencia_train = np.mean(EficienciaTrain)
    eficiencia_val = np.mean(EficienciaVal)
    idc_train = np.std(EficienciaTrain)
    idc_val = np.std(EficienciaVal)
    print('Eficiencia durante el entrenamiento = ' + str(eficiencia_train) + '+-' + str(idc_train))
    print('Eficiencia durante la validación = ' + str(eficiencia_val) + '+-' + str(idc_val))
    return eficiencia_val, idc_val, 0

Realice los experimientos necesarios para llenar la siguiente tabla:

In [137]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
df_types["Eficiencia en validacion"][3] = "0.97633"
df_types["Intervalo de confianza"][3] = "0.01837"
df_types["% de Vectores de Soporte"][3] = "0.2778"
#df_types.sort_index(inplace=True)
for i in df_types.index:
    eficiencia, idc, porc_maquinas = run_through_tests(i[0], i[1], i[2])
    df_types["Eficiencia en validacion"][i] = eficiencia
    df_types["Intervalo de confianza"][i] = idc
    df_types["% de Vectores de Soporte"][i] = porc_maquinas
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

Eficiencia durante el entrenamiento = 0.9930503542602132+-0.004223723406637225
Eficiencia durante la validación = 0.9539153482422209+-0.03872796844095479
Eficiencia durante el entrenamiento = 0.9976860250624457+-0.0007984236624606809
Eficiencia durante la validación = 0.9622957538513002+-0.034566415499159724
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9749280997108298+-0.020473001729313472
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9777370884748747+-0.01637228483712346
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9777370884748747+-0.01637228483712346
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9777370884748747+-0.01637228483712346
Eficiencia durante el entrenamiento = 0.9995378927911276+-0.0008003931643109515
Eficiencia durante la validación = 0.9637308590003759+-0.03199738849423081
Eficiencia durante el entrenamiento = 1.0+-0.0

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [34]:
qgrid_widget.get_changed_df()

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
linear 0.001   0.00                  0.953938              0.0387323   
       0.010   0.00                  0.960938              0.0347002   
       0.100   0.00                  0.973524              0.0226372   
       1.000   0.00                  0.976333              0.0183761   
       10.000  0.00                  0.976333              0.0183761   
       100.000 0.00                  0.976333              0.0183761   
rbf    0.001   0.01                  0.962303              0.0338999   
               0.10                  0.951538              0.0338349   
               1.00                  0.660197               0.055599   
       0.010   0.01                  0.965066              0.0350392   
               0.10                  0.916837              0.0324206   
               1.00                  0.263955              0.0154695   
       0.100   0.01                  0.963661              0.0374674   
               0.10                  0.916837              0.0324206   
               1.00                  0.263955              0.0154695   
       1.000   0.01                  0.981796              0.0222707   
               0.10                  0.878014               0.037433   
               1.00                  0.263955              0.0154695   
       10.000  0.01                  0.987437              0.0128178   
               0.10                  0.872474              0.0375695   
               1.00                  0.261162              0.0106959   
       100.000 0.01                  0.987437              0.0128178   
               0.10                  0.872474              0.0375695   
               1.00                  0.261162              0.0106959   

                     % de Vectores de Soporte  
Kernel C       gamma                           
linear 0.001   0.00                         0  
       0.010   0.00                         0  
       0.100   0.00                         0  
       1.000   0.00                         0  
       10.000  0.00                         0  
       100.000 0.00                         0  
rbf    0.001   0.01                         0  
               0.10                         0  
               1.00                         0  
       0.010   0.01                         0  
               0.10                         0  
               1.00                         0  
       0.100   0.01                         0  
               0.10                         0  
               1.00                         0  
       1.000   0.01                         0  
               0.10                         0  
               1.00                         0  
       10.000  0.01                         0  
               0.10                         0  
               1.00                         0  
       100.000 0.01                         0  
               0.10                         0  
               1.00                         0

*En las tablas el punto separa las cifras decimales, es decir 100.000 es 100 ó 1.000 es 1.